In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from os import listdir
from os.path import isfile, join
from sklearn.model_selection import train_test_split
import scipy
import cv2
from matplotlib import pyplot as plt
import time

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def load_data(file_path):
    data = pd.read_csv(file_path, delim_whitespace = True, header = None)
    return data

In [3]:
data = load_data('.\\driving_dataset\\data.txt')

image_names = data[0].as_matrix()
steering_angles = data[1].apply(lambda x: x * scipy.pi / 180).as_matrix()

image_names = np.apply_along_axis(lambda x: '.\\driving_dataset\\' + x, 0, image_names)

train_test_split = 0.8
total_data = len(image_names)
train_count = int(total_data * train_test_split)
x_train, x_test = image_names[:train_count + 1], image_names[train_count + 1:]
y_train, y_test = steering_angles[:train_count + 1], steering_angles[train_count + 1:]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


In [4]:
%matplotlib inline
train_batch_counter = 0
validation_batch_counter = 0
train_size = len(x_train)
val_size = len(x_test)

def load_and_resize_image(path):
    image = cv2.imread(path)
    cropped_image = image[150:, :]
    resized_image = cv2.resize(cropped_image, dsize=(200, 66), interpolation=cv2.INTER_CUBIC) / 255.0
    return resized_image

def get_train_batched_data(batch_size):
    global train_batch_counter
    end_index = train_batch_counter + batch_size
    start_index = train_batch_counter
    
    if end_index >= train_size:
        end_index = train_size
        train_batch_counter = 0
    else:
        train_batch_counter += batch_size

    x_subset = x_train[start_index: end_index]
    x_batched_data = [load_and_resize_image(path) for path in x_subset]
    
    y_batched_data = y_train[start_index: end_index]
    
    return x_batched_data, y_batched_data

def get_validation_batched_data(batch_size):
    global validation_batch_counter
    start_index = validation_batch_counter
    end_index = validation_batch_counter + batch_size
    
    if end_index >= val_size:
        end_index = val_size
        validation_batch_counter = 0
    else:
        validation_batch_counter += batch_size
    
    x_subset = x_test[start_index: end_index]
    x_batched_data = [load_and_resize_image(path) for path in x_subset]
    
    y_batched_data = y_test[start_index: end_index]
    
    validation_batch_counter += batch_size
    
    return x_batched_data, y_batched_data

In [5]:
start_time = time.time()

x = tf.placeholder(dtype = tf.float32, shape = [None, 66, 200, 3], name = 'x')
y = tf.placeholder(tf.float32, name = 'y')
keep_prob = tf.placeholder(tf.float32, name = 'keep_prob')

conv1_w = tf.Variable(tf.truncated_normal(shape = [5, 5, 3, 24], stddev = 0.1))
conv1_b = tf.Variable(tf.constant(0.1, shape = [24]))
conv1 = tf.nn.conv2d(x, conv1_w, strides = [1, 2, 2, 1], padding = 'VALID') + conv1_b
out1 = tf.nn.relu(conv1)

conv2_w = tf.Variable(tf.truncated_normal(shape = [5, 5, 24, 36], stddev = 0.1))
conv2_b = tf.Variable(tf.constant(0.1, shape = [36]))
conv2 = tf.nn.conv2d(out1, conv2_w, strides = [1, 2, 2, 1], padding = 'VALID') + conv2_b
out2 = tf.nn.relu(conv2)

conv3_w = tf.Variable(tf.truncated_normal(shape = [5, 5, 36, 48], stddev = 0.1))
conv3_b = tf.Variable(tf.constant(0.1, shape = [48]))
conv3 = tf.nn.conv2d(out2, conv3_w, strides = [1, 2, 2, 1], padding = 'VALID') + conv3_b
out3 = tf.nn.relu(conv3)

conv4_w = tf.Variable(tf.truncated_normal(shape = [3, 3, 48, 64], stddev = 0.1))
conv4_b = tf.Variable(tf.constant(0.1, shape = [64]))
conv4 = tf.nn.conv2d(out3, conv4_w, strides = [1, 1, 1, 1], padding = 'VALID') + conv4_b
out4 = tf.nn.relu(conv4)

conv5_w = tf.Variable(tf.truncated_normal(shape = [3, 3, 64, 64], stddev = 0.1))
conv5_b = tf.Variable(tf.constant(0.1, shape = [64]))
conv5 = tf.nn.conv2d(out4, conv5_w, strides = [1, 1, 1, 1], padding = 'VALID') + conv5_b
out5 = tf.nn.relu(conv5)

out5_flattened = tf.reshape(out5, [-1, 1152])

fc1_w = tf.Variable(tf.truncated_normal(shape = [1152, 1164], stddev = 0.1))
fc1_b = tf.Variable(tf.constant(0.1, shape = [1164]))
fc1 = tf.nn.relu(tf.matmul(out5_flattened, fc1_w) + fc1_b)

dp1_out = tf.nn.dropout(fc1, keep_prob)

fc2_w = tf.Variable(tf.truncated_normal(shape = [1164, 100], stddev = 0.1))
fc2_b = tf.Variable(tf.constant(0.1, shape = [100]))
fc2 = tf.nn.relu(tf.matmul(dp1_out, fc2_w) + fc2_b)

dp2_out = tf.nn.dropout(fc2, keep_prob)

fc3_w = tf.Variable(tf.truncated_normal(shape = [100, 50], stddev = 0.1))
fc3_b = tf.Variable(tf.constant(0.1, shape = [50]))
fc3 = tf.nn.relu(tf.matmul(dp2_out, fc3_w) + fc3_b)

dp3_out = tf.nn.dropout(fc3, keep_prob)

fc4_w = tf.Variable(tf.truncated_normal(shape = [50, 10], stddev = 0.1))
fc4_b = tf.Variable(tf.constant(0.1, shape = [10]))
fc4 = tf.nn.relu(tf.matmul(dp3_out, fc4_w) + fc4_b)

dp4_out = tf.nn.dropout(fc4, keep_prob)

fc5_w = tf.Variable(tf.truncated_normal(shape = [10, 1], stddev = 0.1))
fc5_b = tf.Variable(tf.constant(0.1, shape = [1]))

y_pred = tf.multiply(tf.atan(tf.matmul(dp4_out, fc5_w) + fc5_b), 2, name = 'y_pred')

sess = tf.InteractiveSession()
trainable_variables = tf.trainable_variables()
saver = tf.train.Saver()

regularization_param = 0.001
mean_squared_error = tf.reduce_mean(tf.square(tf.subtract(y, y_pred)))
l2_regularization = regularization_param * tf.add_n([tf.nn.l2_loss(param) for param in trainable_variables])
loss = mean_squared_error + l2_regularization

optimizer = tf.train.AdamOptimizer(1e-4)
train_step = optimizer.minimize(loss)

sess.run(tf.initialize_all_variables())

epochs = 30
batch_size = 100
model_save_dir = '.\\save'

for epoch in range(epochs): 
    for i in range(int(len(x_train) / batch_size) + 1):
        x_train_batch, y_train_batch = get_train_batched_data(batch_size)
        sess.run(train_step, feed_dict = {x: x_train_batch, y : y_train_batch, keep_prob: 0.8})  
        
        try:
            if i % 10 == 0:
              x_val_batch, y_val_batch = get_validation_batched_data(batch_size)
              loss_value = loss.eval(feed_dict={x: x_val_batch, y: y_val_batch, keep_prob: 1.0})
              print('Loss after epoch: {} and steps: {} is: {}'.format(epoch, i, loss_value))
        except Exception as e:
            print('-------------------------------------------------------------------------')
            print('Caught exception')
            print(e)
            for index, something in enumerate(x_train_batch):
                print('Index: {}, shape: {}'.format(index, something.shape))
            print('-------------------------------------------------------------------------')
        if i % batch_size == 0:
            checkpoint_path = os.path.join(model_save_dir, "model.ckpt")
            filename = saver.save(sess, checkpoint_path)

time_taken = time.time() - start_time            
print('---------------------------------------------------------------------------')
print('---------------------------------------------------------------------------')   
print('---------------------------------------------------------------------------')   
print('Finished')            
print('Time taken: {} secs'.format(time_taken))
print('---------------------------------------------------------------------------') 
print('---------------------------------------------------------------------------')   
print('---------------------------------------------------------------------------')   

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Loss after epoch: 0 and steps: 0 is: 6.4391188621521
Loss after epoch: 0 and steps: 10 is: 6.152121067047119
Loss after epoch: 0 and steps: 20 is: 6.062077522277832
Loss after epoch: 0 and steps: 30 is: 6.328804016113281
Loss after epoch: 0 and steps: 40 is: 7.898158550262451
Loss after epoch: 0 and steps: 50 is: 5.933653354644775
Loss after epoch: 0 and steps: 60 is: 5.906696796417236
Loss after epoch: 0 and steps: 70 is: 5.8509202003479
Loss after epoch: 0 and steps: 80 is: 5.863655090332031
Loss after epoch: 0 and steps: 90 is: 5.748096466064453
Loss after epoch: 0 and steps: 100 is: 5.681267738342285
Loss after epoch: 0 and steps: 110 is: 5.594322681427002
Loss after epoch: 0 an

Loss after epoch: 2 and steps: 180 is: 3.6313040256500244
Loss after epoch: 2 and steps: 190 is: 3.0802807807922363
Loss after epoch: 2 and steps: 200 is: 2.992717742919922
Loss after epoch: 2 and steps: 210 is: 3.8362863063812256
Loss after epoch: 2 and steps: 220 is: 3.149385929107666
Loss after epoch: 2 and steps: 230 is: 2.920828342437744
Loss after epoch: 2 and steps: 240 is: 2.9054453372955322
Loss after epoch: 2 and steps: 250 is: 2.930568218231201
Loss after epoch: 2 and steps: 260 is: 2.864638090133667
Loss after epoch: 2 and steps: 270 is: 3.272865056991577
Loss after epoch: 2 and steps: 280 is: 3.012212038040161
Loss after epoch: 2 and steps: 290 is: 2.812958002090454
Loss after epoch: 2 and steps: 300 is: 2.7932932376861572
Loss after epoch: 2 and steps: 310 is: 7.620666027069092
Loss after epoch: 2 and steps: 320 is: 2.7610137462615967
Loss after epoch: 2 and steps: 330 is: 2.735536575317383
Loss after epoch: 2 and steps: 340 is: 2.718432903289795
Loss after epoch: 2 and s

Loss after epoch: 4 and steps: 300 is: 1.8237773180007935
Loss after epoch: 4 and steps: 310 is: 1.809503436088562
Loss after epoch: 4 and steps: 320 is: 1.8063386678695679
Loss after epoch: 4 and steps: 330 is: 1.7835965156555176
Loss after epoch: 4 and steps: 340 is: 1.8381422758102417
-------------------------------------------------------------------------
Caught exception
Cannot feed value of shape (0,) for Tensor 'x:0', which has shape '(?, 66, 200, 3)'
Index: 0, shape: (66, 200, 3)
Index: 1, shape: (66, 200, 3)
Index: 2, shape: (66, 200, 3)
Index: 3, shape: (66, 200, 3)
Index: 4, shape: (66, 200, 3)
Index: 5, shape: (66, 200, 3)
Index: 6, shape: (66, 200, 3)
Index: 7, shape: (66, 200, 3)
Index: 8, shape: (66, 200, 3)
Index: 9, shape: (66, 200, 3)
Index: 10, shape: (66, 200, 3)
Index: 11, shape: (66, 200, 3)
Index: 12, shape: (66, 200, 3)
Index: 13, shape: (66, 200, 3)
Index: 14, shape: (66, 200, 3)
Index: 15, shape: (66, 200, 3)
Index: 16, shape: (66, 200, 3)
Index: 17, shape: (

Loss after epoch: 6 and steps: 80 is: 1.8881418704986572
Loss after epoch: 6 and steps: 90 is: 1.440091609954834
Loss after epoch: 6 and steps: 100 is: 1.40189528465271
Loss after epoch: 6 and steps: 110 is: 2.3073501586914062
Loss after epoch: 6 and steps: 120 is: 1.5840544700622559
Loss after epoch: 6 and steps: 130 is: 1.3574107885360718
Loss after epoch: 6 and steps: 140 is: 1.3585588932037354
Loss after epoch: 6 and steps: 150 is: 1.4010016918182373
Loss after epoch: 6 and steps: 160 is: 1.3430451154708862
Loss after epoch: 6 and steps: 170 is: 1.7384819984436035
Loss after epoch: 6 and steps: 180 is: 1.5144575834274292
Loss after epoch: 6 and steps: 190 is: 1.3347569704055786
Loss after epoch: 6 and steps: 200 is: 1.3228232860565186
Loss after epoch: 6 and steps: 210 is: 6.0040764808654785
Loss after epoch: 6 and steps: 220 is: 1.3063708543777466
Loss after epoch: 6 and steps: 230 is: 1.2945899963378906
Loss after epoch: 6 and steps: 240 is: 1.288264274597168
Loss after epoch: 6 

Loss after epoch: 8 and steps: 200 is: 0.9436613321304321
Loss after epoch: 8 and steps: 210 is: 0.941436231136322
Loss after epoch: 8 and steps: 220 is: 0.9400508403778076
Loss after epoch: 8 and steps: 230 is: 0.9213535785675049
Loss after epoch: 8 and steps: 240 is: 0.9836922883987427
-------------------------------------------------------------------------
Caught exception
Cannot feed value of shape (0,) for Tensor 'x:0', which has shape '(?, 66, 200, 3)'
Index: 0, shape: (66, 200, 3)
Index: 1, shape: (66, 200, 3)
Index: 2, shape: (66, 200, 3)
Index: 3, shape: (66, 200, 3)
Index: 4, shape: (66, 200, 3)
Index: 5, shape: (66, 200, 3)
Index: 6, shape: (66, 200, 3)
Index: 7, shape: (66, 200, 3)
Index: 8, shape: (66, 200, 3)
Index: 9, shape: (66, 200, 3)
Index: 10, shape: (66, 200, 3)
Index: 11, shape: (66, 200, 3)
Index: 12, shape: (66, 200, 3)
Index: 13, shape: (66, 200, 3)
Index: 14, shape: (66, 200, 3)
Index: 15, shape: (66, 200, 3)
Index: 16, shape: (66, 200, 3)
Index: 17, shape: (

...

Loss after epoch: 21 and steps: 50 is: 0.5198692083358765
Loss after epoch: 21 and steps: 60 is: 0.3179037272930145
Loss after epoch: 21 and steps: 70 is: 0.2174331694841385
Loss after epoch: 21 and steps: 80 is: 0.4696490466594696
Loss after epoch: 21 and steps: 90 is: 0.42328447103500366
Loss after epoch: 21 and steps: 100 is: 0.20617187023162842
Loss after epoch: 21 and steps: 110 is: 0.20894280076026917
Loss after epoch: 21 and steps: 120 is: 0.25139105319976807
Loss after epoch: 21 and steps: 130 is: 0.21191442012786865
Loss after epoch: 21 and steps: 140 is: 0.5834341049194336
Loss after epoch: 21 and steps: 150 is: 0.4064750671386719
Loss after epoch: 21 and steps: 160 is: 0.2598854601383209
Loss after epoch: 21 and steps: 170 is: 0.251742959022522
Loss after epoch: 21 and steps: 180 is: 4.970211982727051
Loss after epoch: 21 and steps: 190 is: 0.21219977736473083
Loss after epoch: 21 and steps: 200 is: 0.2113317847251892
Loss after epoch: 21 and steps: 210 is: 0.208667293190956

Loss after epoch: 23 and steps: 150 is: 0.19217590987682343
Loss after epoch: 23 and steps: 160 is: 0.1810208559036255
Loss after epoch: 23 and steps: 170 is: 0.1814577728509903
Loss after epoch: 23 and steps: 180 is: 0.1815059632062912
Loss after epoch: 23 and steps: 190 is: 0.18607619404792786
Loss after epoch: 23 and steps: 200 is: 0.18238987028598785
Loss after epoch: 23 and steps: 210 is: 0.23823422193527222
-------------------------------------------------------------------------
Caught exception
Cannot feed value of shape (0,) for Tensor 'x:0', which has shape '(?, 66, 200, 3)'
Index: 0, shape: (66, 200, 3)
Index: 1, shape: (66, 200, 3)
Index: 2, shape: (66, 200, 3)
Index: 3, shape: (66, 200, 3)
Index: 4, shape: (66, 200, 3)
Index: 5, shape: (66, 200, 3)
Index: 6, shape: (66, 200, 3)
Index: 7, shape: (66, 200, 3)
Index: 8, shape: (66, 200, 3)
Index: 9, shape: (66, 200, 3)
Index: 10, shape: (66, 200, 3)
Index: 11, shape: (66, 200, 3)
Index: 12, shape: (66, 200, 3)
Index: 13, shap

Loss after epoch: 24 and steps: 320 is: 0.28094226121902466
Loss after epoch: 24 and steps: 330 is: 0.21591362357139587
Loss after epoch: 24 and steps: 340 is: 0.1790589541196823
Loss after epoch: 24 and steps: 350 is: 0.4060481786727905
Loss after epoch: 24 and steps: 360 is: 0.5039803981781006
Loss after epoch: 25 and steps: 0 is: 0.17584984004497528
Loss after epoch: 25 and steps: 10 is: 0.29233068227767944
Loss after epoch: 25 and steps: 20 is: 1.8647373914718628
Loss after epoch: 25 and steps: 30 is: 0.37292230129241943
Loss after epoch: 25 and steps: 40 is: 0.5889137983322144
Loss after epoch: 25 and steps: 50 is: 0.4020099341869354
Loss after epoch: 25 and steps: 60 is: 0.21695639193058014
Loss after epoch: 25 and steps: 70 is: 0.19556176662445068
Loss after epoch: 25 and steps: 80 is: 5.1406660079956055
Loss after epoch: 25 and steps: 90 is: 0.17083075642585754
Loss after epoch: 25 and steps: 100 is: 0.16651420295238495
Loss after epoch: 25 and steps: 110 is: 0.1686073392629623

Loss after epoch: 27 and steps: 40 is: 0.22453030943870544
Loss after epoch: 27 and steps: 50 is: 0.17078851163387299
Loss after epoch: 27 and steps: 60 is: 0.15485253930091858
Loss after epoch: 27 and steps: 70 is: 0.15621696412563324
Loss after epoch: 27 and steps: 80 is: 0.1551375389099121
Loss after epoch: 27 and steps: 90 is: 0.16018950939178467
Loss after epoch: 27 and steps: 100 is: 0.15287375450134277
Loss after epoch: 27 and steps: 110 is: 0.21200934052467346
-------------------------------------------------------------------------
Caught exception
Cannot feed value of shape (0,) for Tensor 'x:0', which has shape '(?, 66, 200, 3)'
Index: 0, shape: (66, 200, 3)
Index: 1, shape: (66, 200, 3)
Index: 2, shape: (66, 200, 3)
Index: 3, shape: (66, 200, 3)
Index: 4, shape: (66, 200, 3)
Index: 5, shape: (66, 200, 3)
Index: 6, shape: (66, 200, 3)
Index: 7, shape: (66, 200, 3)
Index: 8, shape: (66, 200, 3)
Index: 9, shape: (66, 200, 3)
Index: 10, shape: (66, 200, 3)
Index: 11, shape: (66

Loss after epoch: 28 and steps: 220 is: 0.4837949872016907
Loss after epoch: 28 and steps: 230 is: 0.20262935757637024
Loss after epoch: 28 and steps: 240 is: 0.14705149829387665
Loss after epoch: 28 and steps: 250 is: 0.39750388264656067
Loss after epoch: 28 and steps: 260 is: 0.41661083698272705
Loss after epoch: 28 and steps: 270 is: 0.14130361378192902
Loss after epoch: 28 and steps: 280 is: 0.1497088521718979
Loss after epoch: 28 and steps: 290 is: 0.1707240343093872
Loss after epoch: 28 and steps: 300 is: 0.1433073729276657
Loss after epoch: 28 and steps: 310 is: 0.5337549448013306
Loss after epoch: 28 and steps: 320 is: 0.4893394410610199
Loss after epoch: 28 and steps: 330 is: 0.29209262132644653
Loss after epoch: 28 and steps: 340 is: 0.23761484026908875
Loss after epoch: 28 and steps: 350 is: 5.5597686767578125
Loss after epoch: 28 and steps: 360 is: 0.16526176035404205
Loss after epoch: 29 and steps: 0 is: 0.1752837896347046
Loss after epoch: 29 and steps: 10 is: 0.569114565

In [5]:
# TEST

tf.reset_default_graph()

sess = tf.Session()

loader = tf.train.import_meta_graph('.\\save\\model.ckpt.meta')
loader.restore(sess, '.\\save\\model.ckpt')
graph = tf.get_default_graph()

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from .\save\model.ckpt


In [6]:
x_input = graph.get_tensor_by_name('x:0')
x_keep_prob = graph.get_tensor_by_name('keep_prob:0')
y_pred_out = graph.get_operation_by_name("y_pred")

In [7]:
actual_degrees = [radians * (180/scipy.pi) for radians in y_test.tolist()]

In [8]:
predicted_radians = []
for path in x_test:
    test_image = load_and_resize_image(path)
    result = sess.run(y_pred_out, feed_dict={x_input: [test_image], x_keep_prob: 1.0})
    predicted_radians.append(result)
    
predicted_radians_flattened = np.concatenate(predicted_radians).ravel().tolist()
predicted_degrees = [radians * (180/scipy.pi) for radians in predicted_radians_flattened]

In [9]:
for i in range(len(actual_degrees)):
    actual = actual_degrees[i]
    predicted = predicted_degrees[i]
    print('Actual: {}, Predicted: {}'.format(actual, predicted))

Actual: -10.79, Predicted: -11.46013772544562
Actual: -10.08, Predicted: -9.72508982595259
Actual: -9.380000000000003, Predicted: -8.147184711025686
Actual: -7.559999999999999, Predicted: -5.632357979799718
Actual: -5.949999999999999, Predicted: -6.177476341339692
Actual: -3.23, Predicted: -5.2231404871553355
Actual: -0.81, Predicted: -2.135240923979605
Actual: 1.21, Predicted: 3.1395143574316857
Actual: 3.4300000000000006, Predicted: 4.608818971239591
Actual: 4.44, Predicted: 4.846975830988397
Actual: 5.34, Predicted: 6.473426165040641
Actual: 5.749999999999999, Predicted: 5.749491597265905
Actual: 7.16, Predicted: 9.069672953217651
Actual: 8.27, Predicted: 9.5673432691764
Actual: 9.48, Predicted: 7.969808629114478
Actual: 0.0, Predicted: 1.1617564053165244
Actual: 9.88, Predicted: 11.818666601430582
Actual: 9.979999999999999, Predicted: 11.622559626207964
Actual: 10.389999999999999, Predicted: 10.617106529298741
Actual: 11.6, Predicted: 13.059687652286447
Actual: 11.39, Predicted: 13

Actual: -18.55, Predicted: -19.86356666579392
Actual: -17.65, Predicted: -16.10378551955345
Actual: -16.24, Predicted: -16.33535203019326
Actual: -15.13, Predicted: -14.53196070649869
Actual: -14.32, Predicted: -15.60106788039715
Actual: -13.610000000000001, Predicted: -14.732636632516025
Actual: -12.709999999999999, Predicted: -11.512571065579952
Actual: -11.8, Predicted: -12.806557089421114
Actual: -10.689999999999998, Predicted: -10.010357849815733
Actual: -9.08, Predicted: -9.09379341663008
Actual: -8.07, Predicted: -7.581676591478674
Actual: -7.16, Predicted: -6.024399626039182
Actual: -5.949999999999999, Predicted: -6.836282982053188
Actual: -4.54, Predicted: -4.68428707603602
Actual: -3.73, Predicted: -3.692079135018276
Actual: -2.82, Predicted: -1.2073024508233394
Actual: -2.82, Predicted: -0.9589836430036289
Actual: -2.82, Predicted: -2.904186277191805
Actual: -2.82, Predicted: -3.2279774572331483
Actual: -3.03, Predicted: -4.833642413180445
Actual: -3.73, Predicted: -4.124718

Actual: -1.71, Predicted: -2.3626817699367613
Actual: -1.71, Predicted: -2.304734533744966
Actual: -1.71, Predicted: -2.2203405722539453
Actual: -1.6100000000000003, Predicted: -1.3216297798229224
Actual: -1.6100000000000003, Predicted: -2.3913398748489274
Actual: -1.6100000000000003, Predicted: 0.021350503969219226
Actual: -1.6100000000000003, Predicted: -3.1545963726119797
Actual: -1.41, Predicted: 0.35781577335992387
Actual: -1.41, Predicted: 0.5772211545057313
Actual: -1.21, Predicted: 0.7432448075825517
Actual: -1.11, Predicted: -2.6011497945714
Actual: -0.71, Predicted: -1.7658302420138168
Actual: -0.3, Predicted: 1.2841696016398705
Actual: 0.0, Predicted: 1.5551800407250846
Actual: 0.1, Predicted: -0.45876097446580577
Actual: 0.1, Predicted: -1.2971831089213608
Actual: 0.1, Predicted: -0.5753605601917503
Actual: 0.1, Predicted: -0.9470378228437021
Actual: 0.1, Predicted: -0.41662449585282213
Actual: 0.2, Predicted: -0.32959012126264015
Actual: 0.0, Predicted: -1.6326346331694603

Actual: 21.079999999999995, Predicted: 20.058742530690544
Actual: 20.87, Predicted: 19.5986794366175
Actual: 20.87, Predicted: 22.57914268050147
Actual: 21.180000000000003, Predicted: 20.968283487003053
Actual: 21.68, Predicted: 19.86834415288956
Actual: 21.88, Predicted: 21.038395736687793
Actual: 21.98, Predicted: 21.58923149459538
Actual: 21.98, Predicted: 23.092082819403483
Actual: 21.98, Predicted: 20.22030957392873
Actual: 21.98, Predicted: 21.46558888417391
Actual: 21.98, Predicted: 20.45512612122318
Actual: 22.39, Predicted: 22.927124752193592
Actual: 22.99, Predicted: 22.825719585891807
Actual: 23.189999999999998, Predicted: 24.09295333606874
Actual: 23.189999999999998, Predicted: 24.22252867302545
Actual: 23.189999999999998, Predicted: 22.02034346548061
Actual: 23.5, Predicted: 22.391861536588927
Actual: 23.7, Predicted: 24.098404526009674
Actual: 23.799999999999997, Predicted: 22.64600451734509
Actual: 23.799999999999997, Predicted: 23.938471962660405
Actual: 23.799999999999

Actual: 8.27, Predicted: 7.857380117889412
Actual: 8.27, Predicted: 6.46963577318654
Actual: 8.27, Predicted: 9.082537798529874
Actual: 8.27, Predicted: 8.971506620672137
Actual: 8.27, Predicted: 8.302215957455848
Actual: 8.27, Predicted: 6.674902828318792
Actual: 8.27, Predicted: 6.674378843173742
Actual: 8.27, Predicted: 7.530135507347931
Actual: 8.27, Predicted: 7.361099328465976
Actual: 8.27, Predicted: 8.687957156627519
Actual: 8.27, Predicted: 9.373197093117698
Actual: 8.27, Predicted: 10.086713645192074
Actual: 8.27, Predicted: 8.862533939474268
Actual: 8.27, Predicted: 7.759025075601459
Actual: 8.27, Predicted: 9.281800445933012
Actual: 8.27, Predicted: 9.88457520630285
Actual: 8.27, Predicted: 8.63660352325243
Actual: 8.27, Predicted: 6.7119656880766865
Actual: 8.27, Predicted: 7.196342723618102
Actual: 8.27, Predicted: 6.432984089877383
Actual: 8.27, Predicted: 7.7576865399581365
Actual: 8.27, Predicted: 8.10500996351189
Actual: 8.27, Predicted: 6.866839106096711
Actual: 8.27

...

Actual: 0.4, Predicted: 2.2007615842597987
Actual: 0.4, Predicted: 1.41554378636706
Actual: 0.4, Predicted: 0.9674817833159889
Actual: 0.4, Predicted: -1.191034631683908
Actual: 0.5, Predicted: 0.8462788520173015
Actual: 0.5, Predicted: 1.9813005638699193
Actual: 0.71, Predicted: -0.2774302804168265
Actual: 1.11, Predicted: -0.23436665832830728
Actual: 1.31, Predicted: -0.6667762644404323
Actual: 0.0, Predicted: 1.6151058702629784
Actual: 1.41, Predicted: 0.15289012713416184
Actual: 1.51, Predicted: 2.8666613904873453
Actual: 1.51, Predicted: 1.6538860047555477
Actual: 1.51, Predicted: 2.0568308931497192
Actual: 1.92, Predicted: 2.1196759017755418
Actual: 2.22, Predicted: 0.9102840678610349
Actual: 2.22, Predicted: 2.030096769669265
Actual: 2.22, Predicted: 2.4016276487895936
Actual: 2.22, Predicted: 4.035764774535317
Actual: 2.22, Predicted: 2.229301377292669
Actual: 2.52, Predicted: 1.8410937978860127
Actual: 2.52, Predicted: 3.3964214674168285
Actual: 2.52, Predicted: 0.605560505740

Actual: -137.34, Predicted: -138.9533739429174
Actual: -136.93999999999997, Predicted: -138.63938193820604
Actual: -136.73999999999998, Predicted: -135.22379176719392
Actual: -135.92999999999998, Predicted: -137.04372435934593
Actual: -135.13, Predicted: -135.63151149778204
Actual: -134.72, Predicted: -134.02591928428865
Actual: -134.62, Predicted: -133.0949901851589
Actual: -134.72, Predicted: -136.13440333514504
Actual: -134.82, Predicted: -135.69516854840595
Actual: -134.82, Predicted: -136.29828455249458
Actual: -134.72, Predicted: -135.279148348585
Actual: -134.72, Predicted: -133.63436831959496
Actual: 0.0, Predicted: 1.218660299342388
Actual: -134.72, Predicted: -133.61841735036822
Actual: -134.72, Predicted: -136.4074869447135
Actual: -134.72, Predicted: -135.39486843564762
Actual: -135.13, Predicted: -134.88380749338268
Actual: -135.92999999999998, Predicted: -135.9190184710348
Actual: -138.35, Predicted: -139.70483603804325
Actual: -140.57000000000002, Predicted: -141.2417755

Actual: -16.34, Predicted: -17.441741979462826
Actual: -16.64, Predicted: -18.032437491417546
Actual: -17.04, Predicted: -17.276946971378145
Actual: -17.140000000000004, Predicted: -15.56657198695627
Actual: -17.239999999999995, Predicted: -15.578248044370728
Actual: -17.239999999999995, Predicted: -15.401656992882515
Actual: -17.239999999999995, Predicted: -19.114030299273843
Actual: -17.239999999999995, Predicted: -17.559920723496028
Actual: -17.239999999999995, Predicted: -17.36579771533751
Actual: -17.239999999999995, Predicted: -17.850366385494734
Actual: -17.239999999999995, Predicted: -16.552613811546614
Actual: -17.34, Predicted: -18.87713982000706
Actual: -17.34, Predicted: -18.53054027625908
Actual: -17.34, Predicted: -16.9192046036312
Actual: -17.34, Predicted: -17.226639522537212
Actual: -17.34, Predicted: -17.696884164344265
Actual: -17.34, Predicted: -17.68099491654656
Actual: -17.34, Predicted: -18.52975777147141
Actual: -17.34, Predicted: -17.514749430829234
Actual: -17

Actual: -1.92, Predicted: -1.4912029302923218
Actual: -2.02, Predicted: -3.304212141722097
Actual: -2.32, Predicted: -0.9784423278108973
Actual: -2.52, Predicted: -1.6172217103933515
Actual: -2.92, Predicted: -2.0196413788081085
Actual: -3.13, Predicted: -3.5858106564112853
Actual: -3.13, Predicted: -3.483810787644313
Actual: 0.0, Predicted: -0.45106852669762176
Actual: -3.8299999999999996, Predicted: -3.8494689644496614
Actual: -4.24, Predicted: -4.3248642433983555
Actual: -4.24, Predicted: -3.050388111992196
Actual: -4.24, Predicted: -4.5271394372939495
Actual: -4.24, Predicted: -2.424356429966956
Actual: -4.24, Predicted: -3.2097077777286764
Actual: -4.24, Predicted: -2.7703793601189317
Actual: -4.24, Predicted: -4.596518619220461
Actual: -4.24, Predicted: -2.684263543548555
Actual: -4.24, Predicted: -4.992470091314962
Actual: -4.24, Predicted: -4.473545436631394
Actual: -4.24, Predicted: -6.000532280517881
Actual: -4.24, Predicted: -6.010443030676524
Actual: -4.24, Predicted: -4.95

Actual: 9.78, Predicted: 9.149759665210025
Actual: 9.78, Predicted: 10.17362354917472
Actual: 9.78, Predicted: 7.906091686915994
Actual: 9.380000000000003, Predicted: 7.717268073610738
Actual: 9.08, Predicted: 9.638047580433675
Actual: 8.97, Predicted: 10.408832468793449
Actual: 8.97, Predicted: 9.213650352312346
Actual: 9.08, Predicted: 10.794459554160682
Actual: 9.08, Predicted: 9.26561138802486
Actual: 8.97, Predicted: 9.934277094059102
Actual: 8.97, Predicted: 10.153772085735776
Actual: 8.97, Predicted: 10.304235068573046
Actual: 8.87, Predicted: 9.356286169151009
Actual: 8.87, Predicted: 8.739484530935771
Actual: 0.0, Predicted: 0.594678106480043
Actual: 8.87, Predicted: 7.595228397576973
Actual: 8.87, Predicted: 9.099650197141779
Actual: 8.87, Predicted: 9.285630710801952
Actual: 8.87, Predicted: 9.367852316767703
Actual: 8.87, Predicted: 7.152796772209393
Actual: 8.87, Predicted: 9.007809712894977
Actual: 8.77, Predicted: 10.493084498423858
Actual: 8.77, Predicted: 10.7656398922